In [1]:
# imports
import os
import imp
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

# directories
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')

In [2]:
data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

# Prototyping

In [3]:
def run_full_fit(arch, archname):
    """
    Runs the full fitting experiment, pretraining on later experiments and testing on first.
    Saves data as it goes to avoid eating memory.
    """

    # start training
    trainer = DefaultTrainer(stopthresh=75, print_interval=20)
    net_list = trainer.train_all(architecture=arch, data=data, seed=985227)
    
    # save params
    for i, n in enumerate(net_list):
        fname = '{} {} split agg fit exp 1-4'.format(archname, i)
        n.save_params(os.path.join(paramsdir, fname))
    
    tuner = FineTuner(stopthresh=20)

    for i, n in enumerate(net_list):
        for j in range(5):
            fname = '{} {} agg fit exp 1-4 {} tune fit exp 0'.format(archname, i, j)
            params = L.get_all_param_values(n.net)
            subnet = tuner.train_all(architecture=arch, data=hvhdata, split=j, startparams=params, freeze=True)
            subnet.save_params(os.path.join(paramsdir, fname))
    
    return None

In [ ]:
archkws = {
    'num_filters': 32, 'filter_size': (3, 3),
    'pad': 'same', 
    'pool_size': 2, 'pool': False
}
arch = lambda input_var: multiconvX #archX_oddfilter_variant(input_var, **archkws)
archname = 'multiconvX_lrg' #.format(archkws['num_filters'])
run_full_fit(multiconvX, archname)


Split Number 0
(12362, 2, 4, 9)
Epoch 0 took 18.348s
	training loss:			2.7881
	validation loss:		2.4920
	validation accuracy:		23.68%
	total time elapsed:		18.731s
Epoch 20 took 17.409s
	training loss:			2.0526
	validation loss:		1.9341
	validation accuracy:		41.24%
	total time elapsed:		388.445s
Epoch 40 took 17.645s
	training loss:			2.0307
	validation loss:		1.9146
	validation accuracy:		41.62%
	total time elapsed:		755.861s
Epoch 60 took 17.241s
	training loss:			2.0224
	validation loss:		1.9025
	validation accuracy:		42.37%
	total time elapsed:		1109.079s
Epoch 80 took 16.997s
	training loss:			2.0210
	validation loss:		1.8995
	validation accuracy:		42.49%
	total time elapsed:		1464.037s
Epoch 100 took 17.576s
	training loss:			2.0187
	validation loss:		1.8963
	validation accuracy:		42.55%
	total time elapsed:		1821.586s
Epoch 120 took 18.250s
	training loss:			2.0116
	validation loss:		1.8945
	validation accuracy:		42.85%
	total time elapsed:		2181.796s
Epoch 140 took 18.728s
	t

## Data aggregation

doesn't need run more than once

In [2]:
datafilenames = ['0 (with groups)', '1 (with computer)', '2 (with computer)', '3 (with computer)', '4']
datafilenames = [os.path.join(datadir, fname + '.csv') for fname in datafilenames]
colnames = ['subject', 'color', 'bp', 'wp', 'zet', 'rt']

e0 = pd.read_csv(datafilenames[0], names=colnames+['splitno'])
e1 = pd.read_csv(datafilenames[1], names=colnames)
e2 = pd.read_csv(datafilenames[2], names=colnames)
e3 = pd.read_csv(datafilenames[3], names=colnames+['task', 'taskorder', 'session'])
e4 = pd.read_csv(datafilenames[4], names=colnames+['timecondition'])
Es = [e1, e2, e3, e4]
for i, e in enumerate(Es[1:]):
    e['subject'] = e['subject'] + Es[i-1].loc[Es[i-1]['subject']<1000, 'subject'].max()

A = pd.concat([e[colnames] for e in [e1, e2, e3, e4]])

groups = np.arange(len(A))%5 + 1
np.random.seed(100001)
np.random.shuffle(groups)
A['group'] = groups

A.to_csv(os.path.join(datadir, '1-4.csv'), encoding='ASCII', header=False, index=False)
A.loc[A['subject']<1000, :].to_csv(
    os.path.join(datadir, '1-4 (no computer).csv'), 
    encoding='ASCII', header=False, index=False
)

# Prototyping Tuning

### Pretraining

In [ ]:
trainer = DefaultTrainer(stopthresh=100) # default: 125
net_list = trainer.train_all(architecture=prototype, data=data, seed=985227)

for i, n in enumerate(net_list):
    n.save_params(os.path.join(paramsdir, '{} split agg fit exp 1-4'.format(i)))

In [ ]:
subresults = []
for s in np.unique(Ss):
    idx = np.where(Ss==s)[0]
    res = results_array[:, idx].mean()
    subresults.append(res)
bmvs(np.array(subresults), alpha=.95)

### Finetuning

In [ ]:
tune_results = np.zeros([40, 5, 5])
tuner = FineTuner(stopthresh=25)

for i, n in enumerate(net_list):
    for j in range(5):
        params = L.get_all_param_values(n.net)
        subnet = tuner.train_all(architecture=prototype, data=hvhdata, split=j, startparams=params, freeze=True)
        subnet.save_params(os.path.join(paramsdir, '{} agg fit exp 1-4 {} tune fit exp 0'.format(i, j)))
        
        res = subnet.itemized_test_fn(Xs, ys)
        for s in np.unique(Ss):
            idx = np.where(Ss==s)[0]
            tune_results[s, j, i] = res[idx].mean()

In [ ]:
# class FineTuner(DefaultTrainer):
#     """
#     Trainer to fine tune networks to individual subjects

#     Consider moving freeze, param set functions properly into Network object
#     Abstracting split functions and augment in DefaultTrainer would be good too
#     """

#     def train_all(self, architecture, data, split, seed=None, startparams=None, freeze=True, save_params=False):
#         if seed:
#             np.random.seed(seed)

#         D, groups, Xs, ys, Ss = data
#         num_splits = len(Xs)
#         r = np.tile(np.arange(num_splits), [num_splits, 1])
#         r = (r + r.T) % num_splits

#         starttime = time.time()
#         net = Network(architecture)
#         if startparams:
#             _layers = L.get_all_layers(net.net)
#             L.set_all_param_values(_layers, startparams)
#             convlayer, prelulayer = _layers[1:3]
#             if freeze:
#                 convlayer.params[convlayer.W].remove('trainable')
#                 convlayer.params[convlayer.b].remove('trainable')
#                 prelulayer.params[prelulayer.alpha].remove('trainable')

#         train_idxs = r[split, :3]
#         val_idxs = r[split, 3:4]
#         test_idxs = r[split, 4:]

#         X, y, S = [np.concatenate(np.array(Z)[train_idxs]) for Z in [Xs, ys, Ss]]
#         Xv, yv, Sv = [np.concatenate(np.array(Z)[val_idxs]) for Z in [Xs, ys, Ss]]
#         Xt, yt, St = [np.concatenate(np.array(Z)[test_idxs]) for Z in [Xs, ys, Ss]]
#         X, y = augment((X, y))
#         S = np.concatenate([S, S, S, S])
#         self.train(net, training_data=(X, y), validation_data=(Xv, yv))
#         self.test(net, testing_data=(Xt, yt))
#         time_elapsed = time.time() - starttime

#         return net